In [1]:
import pandas as pd
import numpy as np

In [2]:
f = '../data/001_raw/ENCEVI_2018/encevi.csv'
columnas = ['folio','tipo_viv','total_niv','super_cons',
            'super_ter','tot_cuart','electri','local_com',
            # 'cons_med1','cons_med2','inicia1','mes_inicia1',
            # 'inica2','mes_inicia2',
           ]
encevi = pd.read_csv(f,usecols=columnas)
columnas = encevi.columns
encevi.set_index('folio', inplace=True)



f = '../data/001_raw/ENCEVI_2018/vivienda.csv'
columnas = ['folio','factor_sem','est_socio','region','entidad']
vivienda = pd.read_csv(f,usecols=columnas)
# vivienda = pd.read_csv(f)
vivienda.set_index('folio',inplace=True)

vivienda = vivienda.join(encevi,how='inner')


f = '../data/001_raw/ENCEVI_2018/hogar.csv'
columnas = ['folio','tot_integ']
hogar = pd.read_csv(f,usecols=columnas)
hogar.set_index('folio',inplace=True)
vivienda = vivienda.join(hogar,how='inner')

f = '../data/001_raw/ENCEVI_Hernandez/encevi_result_db_041722.csv'
columnas = ['folioviv','bill_kwh_estimation','bill_days','climate_region_name']
energia = pd.read_csv(f,usecols=columnas,index_col=0)

vivienda = energia.join(vivienda,how='inner')
vivienda.index.names = ['folio']
vivienda.factor_sem.sum()


# Supongamos que tu dataframe se llama vivienda y la columna con el código es 'super_cons'
mapeo_m2 = {
    1: 30,
    2: 55,
    3: 75,
    4: 100,
    5: 150,
    6: 200,
    7: np.nan,
    9: np.nan
}
mapeo_estrato = {
    1: 'Bajo',
    2: 'Medio bajo',
    3: 'Medio alto',
    4: 'Alto'
}
mapeo_climas = {
    "Extremely hot":'Cálido extremoso',
    "Temperate":    'Templado',
    "Tropical":     'Tropical'
}

# Crear la nueva columna "m2" aplicando el diccionario de mapeo
vivienda['m2'] = vivienda['super_cons'].map(mapeo_m2)
vivienda['estrato'] = vivienda['est_socio'].map(mapeo_estrato)
vivienda['climate_region_name'] = vivienda['climate_region_name'].map(mapeo_climas)
vivienda['kwh_mes'] = (vivienda['bill_kwh_estimation'] / vivienda['bill_days']) * 30
vivienda['kwh_m2'] = vivienda['bill_kwh_estimation']/vivienda['m2']
del vivienda['bill_days']
del vivienda['est_socio']
del vivienda['bill_kwh_estimation']
del vivienda['super_cons']
vivienda

,climate_region_name,factor_sem,region,entidad,tipo_viv,total_niv,super_ter,tot_cuart,electri,local_com,tot_integ,m2,estrato,kwh_mes,kwh_m2
folio,,,,,,,,,,,,,,,
1,Templado,373,2,1,1,2,3,11,1,2,1,NaN,Medio alto,55.065630,NaN
2,Templado,373,2,1,1,1,3,6,1,2,4,75.0,Medio alto,128.896444,1.718619
3,Templado,373,2,1,1,2,3,5,1,2,4,NaN,Medio alto,228.645610,NaN
4,Templado,373,2,1,1,1,3,6,1,2,1,100.0,Medio alto,167.110992,1.671110
5,Templado,373,2,1,1,1,3,5,1,2,2,100.0,Medio alto,207.517844,2.075178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28941,Templado,382,2,32,1,1,9,4,1,2,5,NaN,Medio bajo,75.679526,NaN
28943,Templado,382,2,32,1,1,6,3,1,2,4,55.0,Medio bajo,20.844893,0.757996
28945,Templado,382,2,32,1,1,9,4,1,2,5,NaN,Medio bajo,39.244557,NaN


https://www.inegi.org.mx/rnm/index.php/catalog/495/data-dictionary/F1?file_name=vivienda

``` 
Categorías
Valor	Categoría
01	Aguascalientes
02	Baja California
03	Baja California Sur
04	Campeche
05	Coahuila de Zaragoza
06	Colima
07	Chiapas
08	Chihuahua
09	Ciudad de México
10	Durango
11	Guanajuato
12	Guerrero
13	Hidalgo
14	Jalisco
15	México
16	Michoacán de Ocampo
17	Morelos
18	Nayarit
19	Nuevo León
20	Oaxaca
21	Puebla
22	Querétaro
23	Quintana Roo
24	San Luis Potosí
25	Sinaloa
26	Sonora
27	Tabasco
28	Tamaulipas
29	Tlaxcala
30	Veracruz de Ignacio de la Llave
31	Yucatán
32	Zacatecas
```


In [3]:
slp = vivienda[vivienda.entidad ==24].copy()
del slp['climate_region_name']
del slp['region']
del slp['entidad']
del slp['super_ter']
del slp['electri']
del slp['local_com']
del slp['tipo_viv']
slp

,factor_sem,total_niv,tot_cuart,tot_integ,m2,estrato,kwh_mes,kwh_m2
folio,,,,,,,,
20671,1168,2,5,2,NaN,Medio bajo,94.259937,NaN
20672,1168,1,4,3,150.0,Medio bajo,122.549072,1.633988
20676,864,2,4,2,NaN,Medio bajo,81.958682,NaN
20677,864,2,8,1,NaN,Medio bajo,70.316423,NaN
20678,864,3,6,6,NaN,Medio bajo,114.786207,NaN
...,...,...,...,...,...,...,...,...
21598,575,1,3,4,75.0,Bajo,13.412320,0.369584
21599,575,1,4,2,150.0,Bajo,71.638804,0.987024
21600,575,1,7,2,100.0,Bajo,26.533068,0.548350


In [4]:
slp.to_parquet("../data/002_processed/slp.parquet")